# **Algoritmos de Programación Dinámica**

En esta práctica aprenderemos a implementar el algoritmo *Value Iteration* para la resolución de problemas MDPs finitos.

En primer lugar instalamos e importamos las librerías que vamos a utilizar:

In [1]:
!pip install gymnasium pygame
!pip install imageio

In [2]:
import gymnasium as gym
import numpy as np
import imageio
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

### Creación del entorno

Utilizaremos el entorno de **Gymnasium** Frozen Lake. Crea el entorno `'FrozenLake-v1'` con argumentos `is_slippery=False` y `render_mode='rgb_array'` para su visualización en formato imagen.

In [3]:
import gymnasium as gym
env = gym.make('FrozenLake-v1', render_mode="rgb_array",is_slippery=False)

Utilizaremos el entorno sin ningún wrapper aplicado. Elimina los wrappers.

In [4]:
env= env.unwrapped

## Value Iteration

El algoritmo *Value Iteration* pretende obtener la política óptima aproximando $V^*$ aplicando la ecuación de Bellman como una actualización iterativa según la fórmula:

$$
V(s) = max_{a}\sum_{s', r}p(s', r|s, a)[r + γV(s')] \quad \text{para todo } s \in S
$$

Donde el sumatorio $\sum_{s', r}p(s', r|s, a)[r + γv_{k}(s')]$ es $Q(s, a)$.

Por tanto, también podemos escribir $V(s)$ como:
$$V(s) = max_{a} Q(s, a)$$

Una vez obtenido $V ≈ V^* $ se deriva la política óptima $π^* ≈ π = argmax_{a} Q(s,a)$

### Mecanismo de interrupción del algoritmo

Formalmente, el algoritmo solo converge en el límite (infinito), pero en la práctica necesitamos detenerlo antes de este punto. Para ello haremos uso de dos variables, `delta` y `theta`.
- `delta` será un indicador de la estabilidad de los valores de estado entre iteraciones, guardará la mayor diferencia absoluta entre los nuevos valores de estado y los de la anterior iteración.
- `theta` será un umbral de convergencia predefinido.
    - Si `delta` es mayor que `theta` significará que los valores de estado han cambiado significativamente desde la anterior iteración.
    - Si `delta` es menor que `theta` significará que las estimaciones de los valores de estado han "convergido" a una solución estable dentro de la tolerancia `theta` y podremos detener el proceso.

Definimos el umbral de convergencia `theta`:

In [5]:
theta = 0.000001

### Factor de descuento

Definimos el factor de descuento $γ = 0.99$. Es habitual utilizar valores cercanos a 1 para este parámetro.

In [6]:
gamma = 0.99


### Valores de estado

`V` es el vector que almacenará los valores de estado, por tanto su longitud es `(número de estados)`. Lo inicializaremos en 0.

In [7]:
V = np.zeros(env.observation_space.n)



### Ejercicio 1: Obtención de los valores de estado óptimos

Implementa la ecuación de Bellman de forma iterativa para aproximar los valores de estado óptimos $V ≈ V^*$.

Recuerda que `Env.P[s][a]` devuelve una tupla con las transiciones asociadas a una acción `a` en el estado `s` de la forma `(transition_probability, next_state, reward, done)`.

In [9]:
while True:
  delta = 0.0
  for s in range(env.observation_space.n):
      Q = np.zeros(env.action_space.n)

      for a in range(env.action_space.n):
          for transition_probability, next_state, reward, done in env.P[s][a]:
              if done:
                  Q[a] += transition_probability * reward
              else:
                  Q[a] += transition_probability * (reward + gamma * V[next_state])

      v_new = np.max(Q)  # Esto debe ir FUERA del bucle de acciones
      delta = max(delta, np.abs(v_new - V[s]))
      V[s] = v_new
  if delta < theta:
    break


### Ejercicio 2: Extracción de la política óptima

Obtén la política óptima.

In [11]:
# Define una política inicial con probabilidad 0 en todas sus entradas
policy = np.zeros((env.observation_space.n,env.action_space.n))
# Itera sobre todos los estados
for s in range(env.observation_space.n):
    # Inicializa un vector de valores de acción Q para el estado s a 0
    Q = np.zeros((env.observation_space.n))
    # Itera sobre todas las acciones
    for a in range(env.action_space.n):
        # Calcula Q(s, a) según la fórmula con los valores de estado óptimos

        for transition_probability,next_state,reward,done in env.P[s][a] :
          Q[a]+= transition_probability * (reward + gamma * V[next_state])
    # Actualiza la probabilidad correspondiente a la acción óptima en el estado s a 1
    policy[s,np.argmax(Q)] = 1
    print("Politica por estado:",policy[s])

Politica por estado: [0. 1. 0. 0.]
Politica por estado: [0. 0. 1. 0.]
Politica por estado: [0. 1. 0. 0.]
Politica por estado: [1. 0. 0. 0.]
Politica por estado: [0. 1. 0. 0.]
Politica por estado: [1. 0. 0. 0.]
Politica por estado: [0. 1. 0. 0.]
Politica por estado: [1. 0. 0. 0.]
Politica por estado: [0. 0. 1. 0.]
Politica por estado: [0. 1. 0. 0.]
Politica por estado: [0. 1. 0. 0.]
Politica por estado: [1. 0. 0. 0.]
Politica por estado: [1. 0. 0. 0.]
Politica por estado: [0. 0. 1. 0.]
Politica por estado: [0. 0. 1. 0.]
Politica por estado: [1. 0. 0. 0.]


### Ejercicio 3: Evalúa tu implementación

Crea una animación de los pasos que sigue el agente en el entorno Frozen Lake según la política que obtenida con el algoritmo *Value Iteration*.

Es posible que el entorno sin wrappers dé problemas a la hora de ejecutar `env.step(action)`. En tal caso, podemos crear un nuevo entorno para evaluación y no ejecutar `env = env.unwrapped`, ya que para crear la animación ya no necesitamos acceder a `Env.P`.

In [12]:
env.reset()
def run_episode():
    # Resetear el entorno para obtener el estado inicial
    state, _ = env.reset()
    trajectory =[]
    done=False

    while not done:
      action=np.argmax(policy[state])
      next_state, reward, terminated, truncated, _ = env.step(action)
      # El episodio habrá terminado si así lo indican terminated o truncated
      done = terminated or truncated
      # Actualizamos la variable estado al nuevo
      trajectory.append((state,action,reward))
      state = next_state
    return trajectory

def compute_returns(trajectory,gamma):
  returns=[]
  ganancia=0
  x=0
  for state,action,reward in reversed(trajectory):
    ganancia = reward + gamma * ganancia
    returns.insert(0,ganancia)
  return list(returns)


In [13]:
state, _ = env.reset()
frames = []

# Generación del episodio
for _ in range(50):
    # Obtenemos la imagen (frame) del estado actual
    frame = env.render()
    frames.append(frame)

    # Escogemos la acción óptima para el estado actual según la política
    action = np.argmax(policy[state])
    print(f"Estado: {state}, Acción: {action}")
    # Ejecutamos este paso en el entorno
    state, reward, terminated, truncated, info = env.step(action)

    # Si se ha alcanzado la meta detenemos el proceso
    done = terminated or truncated
    if done:
        frame = env.render()
        frames.append(frame)
        break

# Creación de la animación
fig = plt.figure()
img = plt.imshow(frames[0])

def animate(i):
    img.set_data(frames[i])
    return [img]

ani = animation.FuncAnimation(fig, animate, frames=len(frames), interval=500, blit=True)
plt.close(fig)
HTML(ani.to_jshtml())

Estado: 0, Acción: 1
Estado: 4, Acción: 1
Estado: 8, Acción: 2
Estado: 9, Acción: 1
Estado: 13, Acción: 2
Estado: 14, Acción: 2


## Resumen

- *Policy Iteration* y *Value Iteration* son dos algoritmos de programación dinámica para encontrar la política óptima en un MDP.

- *Policy Iteration* alterna entre la evaluación de la política y la mejora de la política hasta que la política converge.

- *Value Iteration* actualiza los valores de estado utilizando la ecuación de optimalidad de Bellman directamente, y una vez que los valores convergen, la política óptima se deriva de ellos.

- Ambos algoritmos deberían encontrar la misma política óptima y los mismos valores de estado óptimos (con ligeras diferencias debido al umbral de convergencia) para un MDP dado, ya que ambos buscan la solución óptima.
